## A Space to Play Around With DataFrames

In [8]:
import pandas as pd
import dash 
import dash   
import dash_core_components as dcc   
import dash_html_components as html  
import plotly.express as px 
import pandas as pd  
from dash.dependencies import Input, Output
import plotly.graph_objects as go 
import yfinance as yf

# borrowing function from database
from database import fetch_all_stock_as_df

In [7]:
!pip install yfinance

  Using cached yfinance-0.1.55.tar.gz (23 kB)
  Using cached multitasking-0.0.9.tar.gz (8.1 kB)
     |████████████████████████████████| 4.5 MB 5.7 MB/s eta 0:00:01
  Created wheel for yfinance: filename=yfinance-0.1.55-py2.py3-none-any.whl size=22616 sha256=3a18d9796e63c13fc47cc04737aef4e3e87c75ac6ac7efc3ccc69f77ec5cbdc3
  Stored in directory: /Users/kevin18le/Library/Caches/pip/wheels/aa/8a/36/59ed4f6fbcb6100967618eeb0696046bf9777a41ac2ff1f9b9
  Created wheel for multitasking: filename=multitasking-0.0.9-py3-none-any.whl size=8366 sha256=ae53eb66b591aa4493972bd6839e00d13acbb0a2dbe37bbe6b71b98e7ca44ede
  Stored in directory: /Users/kevin18le/Library/Caches/pip/wheels/ae/25/47/4d68431a7ec1b6c4b5233365934b74c1d4e665bf5f968d363a
Successfully built yfinance multitasking
  Attempting uninstall: lxml
    Found existing installation: lxml 4.5.0
    Uninstalling lxml-4.5.0:
      Successfully uninstalled lxml-4.5.0


In [18]:
import datetime as dt

In [11]:
df = fetch_all_stock_as_df()

In [15]:
df

,Datetime,SPY,SBUX,AAPL
0,2020-11-30 14:30:00,362.839996,97.900002,117.389999
1,2020-11-30 14:31:00,362.890015,97.750000,117.589996
2,2020-11-30 14:32:00,362.890015,97.650002,118.040001
3,2020-11-30 14:33:00,362.970001,97.639999,118.087700
4,2020-11-30 14:34:00,362.750000,97.605003,118.239998
...,...,...,...,...
1944,2020-12-02 19:19:00,366.149994,98.690002,123.002899
1945,2020-12-02 19:20:00,366.239807,98.690002,123.105003
1946,2020-12-02 19:21:00,366.252014,98.690002,123.040001
1947,2020-12-02 19:22:00,366.304413,98.669998,123.029999


In [34]:
x = df[df['Datetime'].dt.weekday != 7 and df['Datetime'].dt.weekday != 1]
x

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [37]:
# no time zone info
for idx, x in df['Datetime'].iteritems(): 
    if (x.hour >= 9 and x.hour <= 16 and x.minute == 0 and x.second == 0):
        print(x)

2020-11-30 15:00:00
2020-11-30 16:00:00
2020-12-01 15:00:00
2020-12-01 16:00:00
2020-12-02 15:00:00
2020-12-02 16:00:00
2020-12-03 15:00:00
2020-12-03 16:00:00
2020-12-04 15:00:00
2020-12-04 16:00:00


,Datetime,SPY,SBUX,AAPL
0,2020-11-30 14:30:00,362.839996,97.900002,117.389999
1,2020-11-30 14:31:00,362.890015,97.750000,117.589996
2,2020-11-30 14:32:00,362.890015,97.650002,118.040001
3,2020-11-30 14:33:00,362.970001,97.639999,118.087700
4,2020-11-30 14:34:00,362.750000,97.605003,118.239998
...,...,...,...,...
1944,2020-12-02 19:19:00,366.149994,98.690002,123.002899
1945,2020-12-02 19:20:00,366.239807,98.690002,123.105003
1946,2020-12-02 19:21:00,366.252014,98.690002,123.040001
1947,2020-12-02 19:22:00,366.304413,98.669998,123.029999


In [7]:
# Quick test for df

import time
import sched
import logging
import requests
from io import StringIO

import utils
# from database import upsert_bpa

import logging
import pymongo
import expiringdict

import pandas as pd
import yfinance as yf
import numpy as np


####### parameter for testing #######
portfolio = ['TSLA', 'ENPH', 'ZM', 'MRNA', 'PTON', 'BTBT', 'TGT', 'WMT', 'SBUX', 'ABBV']
p = '5d'
i = '1m'


# database info
MAX_DOWNLOAD_ATTEMPT = 5
DOWNLOAD_PERIOD = 10         # second
logger = logging.Logger(__name__)
utils.setup_logger(logger, 'data.log')
client = pymongo.MongoClient()


# Download data for one stock
def download_one(t, p, i):
    data = yf.download(
            tickers = t,
            period = p,
            interval = i,
            group_by = 'ticker',
            auto_adjust = True,
            prepost = False,
            threads = True,
            proxy = None,
            # no progress bar
            progress=False)
    data.reset_index(inplace=True,drop=False)
    data.rename(columns = {'Close':t}, inplace = True) 
    data = data.drop(['Open', 'High','Low','Volume'], axis=1)
    return data


# Download data for a portfolio
def make_portfolio(port, p ,i):
    one = [download_one(port[0],p,i)]
    rst = []
    for t in port[1:]:
        one.append(download_one(t,p,i).drop(['Datetime'], axis=1))
        rst = pd.concat(one, axis=1, sort=False)
    return rst


# Update once
def update_once(port, p ,i):
    df = make_portfolio(port, p, i)
    upsert_stock(df)
    
    
df = make_portfolio(portfolio, p, i)
df

,Datetime,TSLA,ENPH,ZM,MRNA,PTON,BTBT,TGT,WMT,SBUX,ABBV
0,2020-11-30 09:30:00-05:00,603.451172,142.440002,480.513306,145.675003,110.349998,7.1066,179.509995,151.764999,97.900002,105.279999
1,2020-11-30 09:31:00-05:00,605.719971,142.910004,482.959991,148.190002,110.910004,7.0000,179.380005,151.705002,97.750000,105.410004
2,2020-11-30 09:32:00-05:00,601.428711,144.710007,484.265106,147.559998,110.750000,6.7800,179.020004,152.449997,97.650002,105.029999
3,2020-11-30 09:33:00-05:00,598.429993,143.414993,486.116394,146.527206,110.495003,6.8800,179.140106,151.770004,97.639999,105.127197
4,2020-11-30 09:34:00-05:00,600.650024,144.154495,483.480988,147.758194,110.834999,6.9394,178.919998,151.744995,97.605003,105.040001
...,...,...,...,...,...,...,...,...,...,...,...
1937,2020-12-04 15:56:00-05:00,598.685608,NaN,410.399994,152.759995,114.570000,NaN,174.750000,148.639999,102.059998,107.050003
1938,2020-12-04 15:57:00-05:00,598.750000,NaN,410.250000,152.660004,114.470001,NaN,175.000000,148.679993,102.105003,107.120003
1939,2020-12-04 15:58:00-05:00,598.679993,NaN,410.160004,152.796005,114.540001,NaN,174.929993,148.750000,102.224998,107.190002
1940,2020-12-04 15:59:00-05:00,599.000000,NaN,409.989990,152.520004,114.420998,NaN,175.029999,148.779999,102.279999,107.220001
